In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import pandas as pd

import plotly.express as px

import sys
sys.path.append('../')
from utils import get_comments2sentiments_per_minutes, bitcoin_sentiment_scatter_norm


# from sklearn.model_selection 


In [3]:
bitcoin_data = pd.read_csv('bitcoin_data.csv')
# bitcoin_data['bitcoin_norm'] = (bitcoin_data['bitcoin'] - bitcoin_data['bitcoin'].min()) / (bitcoin_data['bitcoin'].max() - bitcoin_data['bitcoin'].min())
comments_data = pd.read_csv('bitcoin_reddit_comments.csv')
sentiments = get_comments2sentiments_per_minutes(comments_data, minutes=10)
sentiments['compound_norm'] = (sentiments['compound_mean'] + 1 ) * 0.5

# Merge sentiments with bitcoin_data on the closest date
sentiments['date'] = pd.to_datetime(sentiments['date'])
bitcoin_data['date'] = pd.to_datetime(bitcoin_data['date'])

# Find the closest date in bitcoin_data for each date in sentiments
sentiments['closest_date'] = sentiments['date'].apply(lambda x: bitcoin_data.iloc[(bitcoin_data['date'] - x).abs().argsort()[:1]]['date'].values[0])

# Merge the dataframes on the closest date
# sentiments = sentiments.merge(bitcoin_data[['date', 'bitcoin_norm']], left_on='closest_date', right_on='date', suffixes=('', '_bitcoin'))
sentiments = sentiments.merge(bitcoin_data[['date', 'bitcoin']], left_on='closest_date', right_on='date', suffixes=('', '_bitcoin'))

# Drop the extra date columns
sentiments.drop(columns=['closest_date', 'date_bitcoin'], inplace=True)
sentiments['comments'] = sentiments[['positive_count','negative_count', 'neutral_count']].sum(axis=1)
daily_people = sentiments.groupby(sentiments['date'].dt.date)['comments'].sum().reset_index()


In [4]:
# scaler = StandardScaler()
scaler = MinMaxScaler(feature_range=(0, 1))  # Default range is [0, 1]
sentiments_scaled = sentiments.copy()
sentiments_scaled[sentiments.columns.difference(['date'])] = scaler.fit_transform(sentiments[sentiments.columns.difference(['date'])])

In [5]:
sentiments_scaled['bitcoin'].corr(sentiments_scaled['comments'], method='pearson')

0.28885967979027793

In [6]:
fig = px.scatter(sentiments_scaled, x='date', y=['bitcoin', 'comments'], title='Bitcoin price vs number of comments')
fig.update_traces(mode='lines')
fig.show()

In [6]:
# sentiments

In [7]:
fig = px.scatter(sentiments_scaled, x='date', y=['bitcoin', 'compound_norm'], title='Bitcoin price vs Sentiment')
fig.update_traces(mode='lines')
fig.show()

In [8]:
sentiments_scaled.date[71] - sentiments_scaled.date[0] 

Timedelta('2 days 00:00:00')

In [9]:
sentiments_scaled['bitcoin'].corr(sentiments_scaled['compound_norm'], method='pearson')
shift_cors = []
for i in range(71):
    shifted_corr = sentiments_scaled['bitcoin'].shift(i).corr(sentiments_scaled['compound_norm'], method='pearson')
    shift_cors.append(shifted_corr)

In [10]:
np.max(shift_cors)

0.10501855014012801

In [11]:
sentiments_scaled.date[3] - sentiments_scaled.date[0] 

Timedelta('0 days 02:20:00')

In [12]:
np.max(shift_cors)

0.10501855014012801

In [13]:
granger_results = grangercausalitytests(sentiments_scaled[['bitcoin', 'compound_norm']], maxlag=6*12)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.4705  , p=0.2264  , df_denom=250, df_num=1
ssr based chi2 test:   chi2=1.4882  , p=0.2225  , df=1
likelihood ratio test: chi2=1.4838  , p=0.2232  , df=1
parameter F test:         F=1.4705  , p=0.2264  , df_denom=250, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.7669  , p=0.4656  , df_denom=247, df_num=2
ssr based chi2 test:   chi2=1.5648  , p=0.4573  , df=2
likelihood ratio test: chi2=1.5600  , p=0.4584  , df=2
parameter F test:         F=0.7669  , p=0.4656  , df_denom=247, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=2.9098  , p=0.0352  , df_denom=244, df_num=3
ssr based chi2 test:   chi2=8.9798  , p=0.0296  , df=3
likelihood ratio test: chi2=8.8229  , p=0.0317  , df=3
parameter F test:         F=2.9098  , p=0.0352  , df_denom=244, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=2.1602  , p=0.0741  

In [14]:
# comp

In [15]:
sentiments_scaled['compound_mean_shifted'] = sentiments_scaled['compound_mean'].shift(3)

In [16]:
from sklearn.ensemble import RandomForestRegressor


In [18]:
train_size = int(len(sentiments_scaled) * 0.7)  # 80% for training, 20% for testing
train, test = sentiments_scaled[:train_size], sentiments_scaled[train_size:]

X_train = train[['compound_mean_shifted', 'comments']]
y_train = train['bitcoin'] 
X_test = test[['compound_mean_shifted', 'comments']]
y_test = test['bitcoin']


# X_train_with_constant = sm.add_constant(X_train)
# modelp = sm.OLS(y_train, X_train_with_constant).fit()
# modelp.summary()
rfr = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10).fit(X_train, y_train)
rfr.score(X_train, y_train)

0.7225806946609105

In [19]:
rfr.score(X_test, y_test)

-5.5601731146571565

,Time,bitcoin
0,2024-12-04T23,28
1,2024-12-05T00,23
2,2024-12-05T01,21
3,2024-12-05T02,48
4,2024-12-05T03,100
...,...,...
164,2024-12-11T19,25
165,2024-12-11T20,28
166,2024-12-11T21,28
167,2024-12-11T22,29


In [ ]:
gbitcoin = pd.read_csv('bitcoin_google_search.csv')#['Category: All categories']
gbitcoin['date'] = pd.to_datetime(gbitcoin['Time'])

# Find the closest date in gbitcoin for each date in sentiments
sentiments['closest_date_gbitcoin'] = sentiments['date'].apply(lambda x: gbitcoin.iloc[(gbitcoin['date'] - x).abs().argsort()[:1]]['date'].values[0])

# Merge the dataframes on the closest date
merged_data = sentiments.merge(gbitcoin, left_on='closest_date_gbitcoin', right_on='date', suffixes=('', '_gbitcoin'))

# Drop the extra date columns
merged_data.drop(columns=['closest_date_gbitcoin', 'date_gbitcoin'], inplace=True)

In [24]:
merged_data

,positive_count,negative_count,neutral_count,compound_mean,date,compound_norm,bitcoin,comments,Time,bitcoin_gbitcoin
0,5,0,4,0.322856,2024-12-04 14:10:00,0.661428,95692,9,2024-12-04T23,28
1,2,1,2,0.132160,2024-12-04 15:20:00,0.566080,95876,5,2024-12-04T23,28
2,1,0,0,0.531900,2024-12-04 16:10:00,0.765950,95568,1,2024-12-04T23,28
3,4,3,3,0.132160,2024-12-04 16:30:00,0.566080,95112,10,2024-12-04T23,28
4,1,0,0,0.334600,2024-12-04 17:20:00,0.667300,94990,1,2024-12-04T23,28
...,...,...,...,...,...,...,...,...,...,...
249,8,1,2,0.484609,2024-12-11 11:10:00,0.742305,98276,11,2024-12-11T11,19
250,0,1,0,-0.381800,2024-12-11 11:20:00,0.309100,98436,1,2024-12-11T11,19
251,2,0,4,0.157267,2024-12-11 11:40:00,0.578633,98351,6,2024-12-11T12,20
252,2,0,3,0.258900,2024-12-11 12:10:00,0.629450,98397,5,2024-12-11T12,20


In [25]:
train_size = int(len(merged_data) * 0.7)  # 80% for training, 20% for testing
train, test = merged_data[:train_size], merged_data[train_size:]

X_train = train[['compound_mean', 'comments', 'bitcoin_gbitcoin']]
y_train = train['bitcoin'] 
X_test = test[['compound_mean', 'comments', 'bitcoin_gbitcoin']]
y_test = test['bitcoin']


# X_train_with_constant = sm.add_constant(X_train)
# modelp = sm.OLS(y_train, X_train_with_constant).fit()
# modelp.summary()
rfr = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10).fit(X_train, y_train)
rfr.score(X_train, y_train)

0.9244913825694889

In [26]:
rfr.score(X_test, y_test)

-3.505759154609099

In [32]:
merged_data_scaled

,positive_count,negative_count,neutral_count,compound_mean,date,compound_norm,bitcoin,comments,Time,bitcoin_gbitcoin
0,5,0,4,0.322856,2024-12-04 14:10:00,0.661428,95692,9,2024-12-04T23,28
1,2,1,2,0.132160,2024-12-04 15:20:00,0.566080,95876,5,2024-12-04T23,28
2,1,0,0,0.531900,2024-12-04 16:10:00,0.765950,95568,1,2024-12-04T23,28
3,4,3,3,0.132160,2024-12-04 16:30:00,0.566080,95112,10,2024-12-04T23,28
4,1,0,0,0.334600,2024-12-04 17:20:00,0.667300,94990,1,2024-12-04T23,28
...,...,...,...,...,...,...,...,...,...,...
249,8,1,2,0.484609,2024-12-11 11:10:00,0.742305,98276,11,2024-12-11T11,19
250,0,1,0,-0.381800,2024-12-11 11:20:00,0.309100,98436,1,2024-12-11T11,19
251,2,0,4,0.157267,2024-12-11 11:40:00,0.578633,98351,6,2024-12-11T12,20
252,2,0,3,0.258900,2024-12-11 12:10:00,0.629450,98397,5,2024-12-11T12,20


In [35]:
# merged_data.drop(columns=['Time'], inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))  # Default range is [0, 1]
merged_data_scaled = merged_data.copy()
merged_data_scaled[merged_data_scaled.columns.difference(['date'])] = scaler.fit_transform(merged_data_scaled[merged_data_scaled.columns.difference(['date'])])
merged_data_scaled.rename(columns={'bitcoin_gbitcoin': 'gbitcoin'}, inplace=True)

In [37]:
fig = px.scatter(merged_data_scaled, x='date', y=['bitcoin', 'gbitcoin'], title='Bitcoin price vs Sentiment')
fig.update_traces(mode='lines')
fig.show()

In [38]:
merged_data_scaled['bitcoin'].corr(merged_data_scaled['gbitcoin'], method='pearson')

0.5465228488149